# Ejercicio final de la semana 1

Para demostrar que estás familiarizado con la API de OpenAI y también con Ollama, crea una herramienta que responda a una pregunta técnica
y la explique. ¡Esta es una herramienta que podrás usar durante el curso!

In [12]:
# imports

import os
import json
import requests
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI


In [8]:
# constantes

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL_GPT = 'deepseek-chat'
MODEL_LLAMA = 'qwen2.5:7b'


In [10]:
# Configura el entorno

load_dotenv()
api_key = os.getenv('DEEPSEEK_API_KEY')

if api_key and api_key[:3]=='sk-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")

openai = OpenAI(api_key=api_key, base_url="https://api.deepseek.com")

La clave de API parece buena


In [11]:
# Aquí está la pregunta; escribe sobre esto para preguntar algo nuevo

system_prompt = "Necesito que actues como un programador experto en Python. \
Responde lo más detallado posible, dando ejemplos. \
Responde en formato markdown.\n"

user_prompt = """
Explica qué hace este código y por qué:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [ ]:
# Haz que deepseek responda, con streaming

def python_expert_cloud():
    # Llamada al modelo con streaming habilitado
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True  # Habilitar streaming
    )

    # Inicializar variables para mostrar el contenido progresivamente
    response = ""
    display_handle = display(Markdown(""), display_id=True)

    # Procesar cada fragmento de la respuesta
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

    return response


# Llamada a la función
final_response = python_expert_cloud()
print("Respuesta final completa:")
print(final_response)


# Análisis del código: `yield from {book.get("author") for book in books if book.get("author")}`

Este código utiliza varias características avanzadas de Python. Vamos a desglosarlo paso a paso:

## Descomposición del código

### 1. **Comprehension de conjuntos (Set Comprehension)**
python
{book.get("author") for book in books if book.get("author")}


Esta parte crea un **conjunto** (set) que contiene:
- Los valores de `book.get("author")` para cada libro en la lista `books`
- Solo incluye los libros donde `book.get("author")` no es `None` ni `False`

### 2. **Función `get()` de diccionarios**
python
book.get("author")

- Devuelve el valor de la clave "author" si existe
- Devuelve `None` si la clave no existe (en lugar de lanzar `KeyError`)
- Es más seguro que `book["author"]`

### 3. **Condición `if book.get("author")`**
- Filtra los libros donde el autor existe y no es un valor "falsy"
- Excluye: `None`, `""`, `0`, `False`, `[]`, etc.

### 4. **`yield from`**
python
yield from conjunto

- Es un **generador** que produce cada elemento del conjunto uno por uno
- Equivale a:
python
for autor in conjunto:
    yield autor


## Ejemplo completo

python
def obtener_autores(books):
    yield from {book.get("author") for book in books if book.get("author")}

# Datos de ejemplo
libros = [
    {"title": "Libro 1", "author": "Autor A"},
    {"title": "Libro 2", "author": "Autor B"},
    {"title": "Libro 3", "author": "Autor A"},  # Duplicado
    {"title": "Libro 4", "author": ""},         # Autor vacío (excluido)
    {"title": "Libro 5"},                       # Sin autor (excluido)
    {"title": "Libro 6", "author": "Autor C"},
]

# Uso del generador
generador = obtener_autores(libros)

# Convertir a lista para ver los resultados
autores_unicos = list(generador)
print(autores_unicos)  # ['Autor A', 'Autor B', 'Autor C']


## ¿Qué hace exactamente?

1. **Elimina duplicados**: Al usar un conjunto, autores repetidos aparecen solo una vez
2. **Filtra valores inválidos**: Excluye libros sin autor o con autor "falsy"
3. **Genera elementos uno por uno**: Es eficiente en memoria para grandes conjuntos de datos

## Ventajas de este enfoque

### Memoria eficiente
python
# Para millones de libros, el generador es más eficiente
def procesar_grandes_datos():
    for autor in obtener_autores(millones_de_libros):
        procesar(autor)  # Procesa uno a la vez


### Código conciso
Equivalente más verboso:
python
def obtener_autores_verbose(books):
    autores = set()
    for book in books:
        autor = book.get("author")
        if autor:  # Si autor existe y no es falsy
            autores.add(autor)
    
    for autor in autores:
        yield autor


## Casos de uso típicos

### 1. API que devuelve autores únicos
python
@app.route('/api/autores')
def obtener_autores_api():
    libros = obtener_libros_de_bd()
    return jsonify(list(obtener_autores(libros)))


### 2. Procesamiento por lotes
python
for autor in obtener_autores(libros):
    enviar_correo_promocional(autor)


## Consideraciones importantes

### Valores "falsy" que podrían ser válidos
python
# Si 0 es un autor válido (poco común)
libros = [{"author": 0}]  # Sería excluido por la condición if


### Alternativa para incluir todos los valores
python
# Incluir incluso valores "falsy" pero excluir None
yield from {book.get("author") for book in books if "author" in book}


## Conclusión

Este código es un ejemplo elegante de:
- **Programación funcional** con comprehensions
- **Generadores** para eficiencia de memoria
- **Manejo seguro** de diccionarios con `get()`
- **Eliminación de duplicados** automática con sets

Es particularmente útil cuando trabajas con grandes volúmenes de datos y necesitas procesar valores únicos de manera eficiente.

Respuesta final completa:
# Análisis del código: `yield from {book.get("author") for book in books if book.get("author")}`

Este código utiliza varias características avanzadas de Python. Vamos a desglosarlo paso a paso:

## Descomposición del código

### 1. **Comprehension de conjuntos (Set Comprehension)**
python
{book.get("author") for book in books if book.get("author")}


Esta parte crea un **conjunto** (set) que contiene:
- Los valores de `book.get("author")` para cada libro en la lista `books`
- Solo incluye los libros donde `book.get("author")` no es `None` ni `False`

### 2. **Función `get()` de diccionarios**
python
book.get("author")

- Devuelve el valor de la clave "author" si existe
- Devuelve `None` si la clave no existe (en lugar de lanzar `KeyError`)
- Es más seguro que `book["author"]`

### 3. **Condición `if book.get("author")`**
- Filtra los libros donde el autor existe y no es un valor "falsy"
- Excluye: `None`, `""`, `0`, `False`, `[]`, etc.

### 4. **`yield from`*

In [ ]:
# Haz que qwen2.5 responda, con streaming

def python_expert_local():
    messages = [
        {
            "role": "user",
            "content": system_prompt + user_prompt
        }
    ]

    payload = {
        "model": MODEL_LLAMA,
        "messages": messages,
        "stream": True
    }

    # Hacer la petición con stream=True en requests
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS, stream=True)

    # Inicializar variables para mostrar el contenido progresivamente
    full_response = ""
    display_handle = display(Markdown(""), display_id=True)

    # Procesar cada línea de la respuesta streaming
    for line in response.iter_lines():
        if line:
            # Decodificar la línea
            line_str = line.decode('utf-8')
            
            # Parsear el JSON de cada línea
            try:
                chunk = json.loads(line_str)
                
                # Extraer el contenido del mensaje
                if 'message' in chunk and 'content' in chunk['message']:
                    content = chunk['message']['content']
                    full_response += content
                    
                    # Limpiar formato si es necesario
                    clean_response = full_response.replace("```markdown", "").replace("```", "")
                    
                    # Actualizar el display progresivamente
                    update_display(Markdown(clean_response), display_id=display_handle.display_id)
                    
            except json.JSONDecodeError:
                continue

    return full_response


# Llamada a la función
final_response = python_expert_local()
print("\nRespuesta final completa:")
print(final_response)


# Explicación del Código

El código proporcionado es una línea de Python que se usa para obtener los autores únicos de un conjunto de libros. Vamos a desglosar y explicar cada parte:

## Parte 1: `book.get("author")`

- **`book`:** Es un diccionario representando un libro. Suponemos que este diccionario contiene información sobre el libro, incluyendo posiblemente su título, autor, publicación, etc.
  
- **`.get("author")`:** Este método intenta obtener el valor asociado con la clave `"author"`. Si `"author"` no existe en el diccionario `book`, `.get()` devolverá `None` (o un valor alternativo que se pueda especificar como segundo argumento, pero en este caso no lo hacemos).

## Parte 2: `{book.get("author") for book in books}`

- **`{... for ...}`:** Esta es una notación de conjunto (set comprehension) en Python. Un conjunto es un tipo de dato que no permite elementos duplicados.
  
- **`for book in books`:** Este ciclo `for` itera sobre cada libro en la colección `books`, que se asume es una lista o cualquier otro iterador de libros.

- **`book.get("author")`:** Para cada libro, intentamos obtener su autor. Si el autor no existe para un libro determinado, `.get()` devolverá `None`.

En resumen, esta línea crea un conjunto donde todos los elementos son autores (o `None`, si no hay autor definido).

## Parte 3: `yield from ...`

- **`yield from`:** Este es un elemento importante en este contexto. El código original podría ser parte de una función generadora o un coroutine. Sin embargo, la notación `yield from` se utiliza para delegar la responsabilidad de producir valores a otro iterador.

Si el código completo fuera:

python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}


Entonces:

- **`get_authors`:** Es una función que toma una lista de libros y devuelve un generador que produce autores únicos.
  
- **`yield from ...`:** Este `yield from` delega la responsabilidad de producir valores a la comprensión de conjunto. Esto es útil porque la comprensión de conjunto no funciona en sí misma como una función generadora, pero si le decimos a Python que `yield from` lo haga, podemos obtener los autores uno por uno.

### Ejemplo

Supongamos tenemos la siguiente lista de libros:

python
books = [
    {"title": "Python Crash Course", "author": "Eric Matthes"},
    {"title": "Clean Code", "author": None},
    {"title": "Refactoring: Improving the Design of Existing Code", "author": "Martin Fowler"}
]


Llamamos a la función `get_authors`:

python
def get_authors(books):
    yield from {book.get("author") for book in books if book.get("author")}

authors = list(get_authors(books))
print(authors)


El resultado será:

python
["Eric Matthes", "Martin Fowler"]


Como se puede ver, el autor `None` no fue incluido en la lista final de autores.

---

Espero que esta explicación te haya sido útil. Si tienes más preguntas o necesitas ejemplos adicionales, ¡no dudes en preguntar!


Respuesta final completa:
```markdown
# Explicación del Código

El código proporcionado es una línea de Python que se usa para obtener los autores únicos de un conjunto de libros. Vamos a desglosar y explicar cada parte:

## Parte 1: `book.get("author")`

- **`book`:** Es un diccionario representando un libro. Suponemos que este diccionario contiene información sobre el libro, incluyendo posiblemente su título, autor, publicación, etc.
  
- **`.get("author")`:** Este método intenta obtener el valor asociado con la clave `"author"`. Si `"author"` no existe en el diccionario `book`, `.get()` devolverá `None` (o un valor alternativo que se pueda especificar como segundo argumento, pero en este caso no lo hacemos).

## Parte 2: `{book.get("author") for book in books}`

- **`{... for ...}`:** Esta es una notación de conjunto (set comprehension) en Python. Un conjunto es un tipo de dato que no permite elementos duplicados.
  
- **`for book in books`:** Este ciclo `for` itera sobre cada libr